In [46]:
import pandas as pd
from transformers import AutoModel, AutoTokenizer
from torch import nn
import torch.nn.functional as F
from sklearn.metrics import accuracy_score, f1_score
from torch.utils.data import DataLoader, Dataset

In [47]:
data = pd.read_excel("/content/drive/MyDrive/젯봇/data.xlsx", engine="openpyxl")#.iloc[:1000]

In [48]:
#data.shape

In [49]:
all_indices = data.index

In [50]:
#len(all_indices)

In [51]:
train_size = int(0.8 * len(all_indices))
val_size = len(all_indices) - train_size

train_idx, val_idx = all_indices[:int(0.8 * len(all_indices))], all_indices[int(0.8*len(all_indices)):]

In [52]:
# 인덱스는 0부터 시작하니까 클래스는 +1을 해줘야한다.
num_classes = max(data["label_idx"].tolist())+1

In [53]:
#train_idx, val_idx

In [54]:
import torch
from torch.utils.data import DataLoader, Dataset, random_split
from transformers import BertForSequenceClassification, BertTokenizer, AdamW, BertModel
from transformers import get_constant_schedule_with_warmup, get_linear_schedule_with_warmup
from tqdm.auto import tqdm

tokenizer = AutoTokenizer.from_pretrained("klue/roberta-small")
model = AutoModel.from_pretrained("klue/roberta-small", num_classes)

Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-small and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [55]:
# 데이터셋 클래스 정의
class TextClassificationDatasetBatch(Dataset):
    def __init__(self, df, tokenizer, batch_size=32):
        self.tokenizer = tokenizer
        self.sentences = df["sentence"].tolist()
        self.labels = df["label_idx"].tolist()

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        # to_return = {
        #     "sentence_encoded":self.encodings[idx],
        #     "label":self.labels[idx]
        # }
        return (self.sentences[idx], self.labels[idx])

In [56]:
train_data = data.iloc[train_idx]
val_data = data.iloc[val_idx]

In [57]:
train_dataset = TextClassificationDatasetBatch(train_data, tokenizer)
val_dataset = TextClassificationDatasetBatch(val_data, tokenizer)

In [58]:
#train_dataset[0]

In [59]:
def collate_bert_tokenizer(samples):
  sentences, labels = zip(*samples)
  labels = torch.tensor(labels, dtype = torch.long)
  encodings = tokenizer(sentences, return_tensors="pt", padding = True, truncation = True, max_length=512)
  return encodings, labels

In [60]:
batch_size = 64
train_dataloader = DataLoader(train_dataset, batch_size = batch_size, shuffle=True, collate_fn = collate_bert_tokenizer)
val_dataloader = DataLoader(val_dataset, batch_size = batch_size, shuffle=False, collate_fn = collate_bert_tokenizer)

In [61]:
#sentences = data["sentence"].tolist()
#tokenized = tokenizer(sentences, return_tensors="pt", padding=True, truncation=True, max_length=512)
#output = model(**tokenized)``

In [62]:
#print(tokenized["input_ids"].shape) # 43은 각 문장의 토큰 수 `

In [63]:
#model(**tokenized)["pooler_output"].shape # 768은 고차원벡터 / 768차원 벡터라는 뜻

In [64]:
#tokenized.keys()

In [65]:
#model(input_ids=tokenized["input_ids"], token_type_ids=tokenized["token_type_ids"], attention_mask=tokenized["attention_mask"])

In [66]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [77]:
class CustomModel(nn.Module):
    def __init__(self, bert_model, num_classes):
        super(CustomModel, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model)
        self.dropout = nn.Dropout(0.1)
        self.fc = nn.Linear(self.bert.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        dropout_output = self.dropout(pooled_output)
        return self.fc(dropout_output)

model = CustomModel("klue/roberta-small", num_classes).to(device)

epochs = 50
optimizer = AdamW(model.parameters(),lr=5e-5)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(train_dataloader) * epochs)

You are using a model of type roberta to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of BertModel were not initialized from the model checkpoint at klue/roberta-small and are newly initialized: ['encoder.layer.3.attention.output.LayerNorm.bias', 'encoder.layer.1.intermediate.dense.weight', 'encoder.layer.2.attention.output.dense.bias', 'encoder.layer.4.attention.output.LayerNorm.bias', 'encoder.layer.4.attention.self.key.bias', 'encoder.layer.3.attention.self.key.bias', 'encoder.layer.5.output.LayerNorm.bias', 'encoder.layer.1.attention.self.key.bias', 'encoder.layer.2.intermediate.dense.weight', 'encoder.layer.2.attention.self.value.weight', 'encoder.layer.5.output.LayerNorm.weight', 'embeddings.LayerNorm.bias', 'encoder.layer.5.attention.self.query.bias', 'encoder.layer.2.attention.output.dense.weight', 'encoder.layer.5.attention.output.LayerNorm.bias', 'encoder.layer.2.attention.output.LayerNorm.bias', '

In [68]:
#import torch.optim as optim

In [69]:
#num_train_samples = len(train_dataloader.dataset)
#print(f"Number of samples in the training dataset: {num_train_samples}")

In [70]:
#num_val_samples = len(val_dataloader.dataset)
#print(f"Number of samples in the validation dataset: {num_val_samples}")

In [71]:
# loss_f = nn.CrossEntropyLoss()

# epochs = 4

# for epoch in range(epochs):
#     print(f"Epoch {epoch + 1}/{epochs}")
#     print('-' * 10)

#     # 모델을 학습 모드로 설정
#     custom_model.train()
#     total_loss = 0.0

#     for i, (_input, _label) in enumerate(tqdm(train_dataloader, desc=f"Epoch {epoch + 1}")):
#         _label = torch.tensor(F.one_hot(torch.tensor(_label, dtype=torch.long), num_classes=num_classes), dtype=torch.float).squeeze()

#         outputs = custom_model(_input)
#         loss = loss_f(outputs, _label.argmax(dim=1).long())
#         loss.backward()  # 역전파 수행
#         optimizer.step()
#         optimizer.zero_grad()

#         total_loss += loss.item()

#     average_loss = total_loss / len(train_dataloader)
#     print(f"Epoch {epoch + 1} - Average Loss : {average_loss:.4f}")

#     # 모델 평가
#     custom_model.eval()
#     all_predictions = []
#     all_labels = []

#     with torch.no_grad():
#         for _input, _label in tqdm(val_dataloader, desc=f"Evaluation Epoch {epoch + 1}"):
#             _label = torch.tensor(F.one_hot(torch.tensor(_label, dtype=torch.long), num_classes=num_classes), dtype=torch.float).squeeze()

#             output = custom_model(_input)
#             predictions = output.argmax(dim=1)

#             all_predictions.extend(predictions.cpu().numpy())
#             all_labels.extend(_label.argmax(dim=1).cpu().numpy())

#         f1 = f1_score(all_labels, all_predictions, average='micro')
#         accuracy = accuracy_score(all_labels, all_predictions)

#         print(f"Epoch {epoch + 1} - F1 Score: {f1:.4f}, Accuracy: {accuracy:.4f}")

In [78]:
import torch
from torch import nn

class_counts = data['label_idx'].value_counts()
max_label = data['label_idx'].max()
class_weights = [class_counts.get(i, class_counts.mean()) for i in range(max_label + 1)]

class_weights_tensor = torch.tensor(class_weights, dtype=torch.float).to(device)
loss_f = nn.CrossEntropyLoss(weight=class_weights_tensor)

# epochs = 4

for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in tqdm(train_dataloader, desc=f"Train Epoch {epoch + 1}"):
        optimizer.zero_grad()

        input_ids = batch[0]['input_ids'].to(device)
        attention_mask = batch[0]['attention_mask'].to(device)
        labels = batch[1].to(device)

        outputs = model(input_ids, attention_mask)
        loss = loss_f(outputs, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()

        total_loss += loss.item()

    average_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch + 1} - Average Loss : {average_loss:.4f}")

    # 모델 평가
    model.eval()
    all_predictions = []
    all_labels = []

    with torch.no_grad():
        for batch in tqdm(val_dataloader, desc=f"Evaluation Epoch {epoch + 1}"):
          input_ids = batch[0]['input_ids'].to(device)
          attention_mask = batch[0]['attention_mask'].to(device)
          labels = batch[1].to(device)

          outputs = model(input_ids, attention_mask)
          predictions = outputs.argmax(dim=1)

          all_predictions.extend(predictions.cpu().numpy())
          all_labels.extend(labels.cpu().numpy())

        f1 = f1_score(all_labels, all_predictions, average='micro')
        accuracy = accuracy_score(all_labels, all_predictions)

        print(f"Epoch {epoch + 1} - F1 Score: {f1:.4f}, Accuracy: {accuracy:.4f}")

Train Epoch 1:   0%|          | 0/197 [00:00<?, ?it/s]

Epoch 1 - Average Loss : 0.5777


Evaluation Epoch 1:   0%|          | 0/50 [00:00<?, ?it/s]

Epoch 1 - F1 Score: 0.2463, Accuracy: 0.2463


Train Epoch 2:   0%|          | 0/197 [00:00<?, ?it/s]

...

Epoch 49 - F1 Score: 0.5512, Accuracy: 0.5512


Train Epoch 50:   0%|          | 0/197 [00:00<?, ?it/s]

Epoch 50 - Average Loss : 0.0212


Evaluation Epoch 50:   0%|          | 0/50 [00:00<?, ?it/s]

Epoch 50 - F1 Score: 0.5515, Accuracy: 0.5515
